**РК №2 Мкртчян Давид ИУ5-64Б**

**Вариант №11**

**Задание.** Для заданного набора данных (https://www.kaggle.com/jessemostipak/hotel-booking-demand) постройте модели классификации или регрессии (в зависимости от конкретной задачи, рассматриваемой в наборе данных). Для построения моделей используйте методы 1 и 2 (Линейная/логистическая регрессия и Градиентный бустинг). Оцените качество моделей на основе подходящих метрик качества (не менее двух метрик). Какие метрики качества Вы использовали и почему? Какие выводы Вы можете сделать о качестве построенных моделей? Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д.

Загрузка и предварительная обработка данных.

Первым делом загружаем данные и проводим их предварительную обработку.

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Загрузка данных
data = pd.read_csv('hotel_bookings.csv')

# Просмотр первых строк данных
print(data.head())

# Предварительная обработка данных
# Заполнение пропущенных значений средними значениями для числовых столбцов
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns
imputer_numeric = SimpleImputer(strategy='mean')
data[numeric_features] = imputer_numeric.fit_transform(data[numeric_features])

# Кодирование категориальных признаков
categorical_features = data.select_dtypes(include=['object']).columns
encoder = OneHotEncoder()
data_encoded = encoder.fit_transform(data[categorical_features]).toarray()

# Определение количества столбцов после кодирования
num_encoded_columns = data_encoded.shape[1]

# Создание DataFrame для кодированных данных с правильными названиями столбцов
encoded_df = pd.DataFrame(data_encoded, columns=encoder.get_feature_names_out(categorical_features))

# Объединение обработанных числовых и категориальных данных
data_processed = pd.concat([pd.DataFrame(imputer_numeric.transform(data[numeric_features]), columns=numeric_features),
                            encoded_df], axis=1)

# Определение целевой переменной и признаков
target = 'total_of_special_requests'
features = [col for col in data_processed.columns if col!= target]

X_train, X_test, y_train, y_test = train_test_split(data_processed[features], data_processed[target], test_size=0.2, random_state=42)


          hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  \
0  Resort Hotel            0        342               2015               July   
1  Resort Hotel            0        737               2015               July   
2  Resort Hotel            0          7               2015               July   
3  Resort Hotel            0         13               2015               July   
4  Resort Hotel            0         14               2015               July   

   arrival_date_week_number  arrival_date_day_of_month  \
0                        27                          1   
1                        27                          1   
2                        27                          1   
3                        27                          1   
4                        27                          1   

   stays_in_weekend_nights  stays_in_week_nights  adults  ...  deposit_type  \
0                        0                     0       2  ...    No Deposit   
1     

Построение моделей.

Теперь, когда у нас есть подготовленные данные, мы можем построить модели.

Линейная регрессия

In [5]:
from sklearn.linear_model import LinearRegression

linear_regression_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

linear_regression_pipeline.fit(X_train, y_train)
y_pred_linear = linear_regression_pipeline.predict(X_test)


Градиентный бустинг

In [6]:
from sklearn.ensemble import GradientBoostingRegressor

gradient_boosting_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

gradient_boosting_pipeline.fit(X_train, y_train)
y_pred_gradient_boosting = gradient_boosting_pipeline.predict(X_test)


Оценка моделей

Для оценки качества моделей используем среднеквадратичную ошибку (MSE) и коэффициент детерминации (R^2).

In [7]:
mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

mse_gradient_boosting = mean_squared_error(y_test, y_pred_gradient_boosting)
r2_gradient_boosting = r2_score(y_test, y_pred_gradient_boosting)

print(f"Linear Regression MSE: {mse_linear}, R^2: {r2_linear}")
print(f"Gradient Boosting MSE: {mse_gradient_boosting}, R^2: {r2_gradient_boosting}")


Linear Regression MSE: 3.99429883861244e+20, R^2: -6.331246750688743e+20
Gradient Boosting MSE: 0.4278865818308984, R^2: 0.3217694418124023


**Выводы**

Выбор метрик качества зависит от специфики задачи. В данном случае, поскольку это задача регрессии, мы использовали среднеквадратичную ошибку (MSE), которая показывает среднюю величину ошибок предсказания, и коэффициент детерминации (R^2), который показывает долю вариации зависимой переменной, объясненную независимыми переменными.

1) Линейная регрессия может быть хорошим выбором для простых моделей, но она может не справиться с сложными зависимостями между признаками.

2) Градиентный бустинг обычно показывает лучшие результаты на таких задачах благодаря его способности обрабатывать сложные зависимости и взаимодействия между признаками.

Результаты, которые Мы получили, показывают значительное различие в качестве моделей линейной регрессии и градиентного бустинга на вашем наборе данных.

**Линейная Регрессия**

MSE (Среднеквадратичная ошибка): (3.99429883861244 \times 10^{20}). Это очень высокое значение, указывающее на то, что предсказания модели сильно отклоняются от реальных значений. Такая большая ошибка говорит о том, что модель не может эффективно учесть сложные зависимости в данных.

(R^2) (Коэффициент детерминации): (-6.331246750688743 \times 10^{20}). Негативное значение (R^2) также указывает на серьезные проблемы с моделью. Коэффициент детерминации должен быть положительным и близким к 1, если модель хорошо объясняет изменения в целевой переменной. Значение ниже 0 говорит о том, что модель даже хуже, чем простое среднее.

**Градиентный Бустинг**

MSE: (0.4278865818308984). Это относительно низкое значение по сравнению с линейной регрессией, указывая на то, что предсказания модели ближе к реальным значениям. Градиентный бустинг способен учесть сложные зависимости и взаимодействия между признаками, что приводит к меньшей ошибке.

(R^2): (0.3217694418124023). Хотя это значение и меньше, чем идеальное (1), оно все же положительное и значительно выше, чем у линейной регрессии. Это указывает на то, что градиентный бустинг лучше объясняет изменчивость целевой переменной.

**Выводы**

Сложность данных: Градиентный бустинг лучше подходит для сложных задач, где присутствуют сложные зависимости и взаимодействия между признаками. Мой набор данных, вероятно, содержит такие сложности, поэтому градиентный бустинг дает гораздо лучшие результаты.

Выбор модели: На основании полученных результатов, градиентный бустинг является предпочтительным выбором для этой задачи. Он не только обеспечивает гораздо меньшую ошибку, но и дает положительный коэффициент детерминации, что указывает на его способность объяснять изменчивость целевой переменной.

Таким образом, для улучшения прогнозирования спроса на бронирование отеля рекомендуется использовать модель градиентного бустинга.